## Load CSV in Spark and save as Parquet, from standard or HDFS filesystems

This notebook show how to load a CSV file with Spark, and store the data as Parquet. This is fine to save small datasets. If 'large' datasets (e.g. 10s of MB), store data on HDFS and use Impala to import. See http://mlg.ulb.ac.be/w/Impala

### Import

In [1]:
import time
import os
import sys
import numpy as np
import pandas as pd
import string
import random

### Start Spark session

Note: Spark should be installed, with binaries available from PATH, and PYTHONPATH set. See
* http://mlg.ulb.ac.be/w/Installation:_Spark_standalone_and_IPython
* http://mlg.ulb.ac.be/w/Get_started:_Jupyter_notebook_and_Spark_UI


In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"

from pyspark.sql import SparkSession

#Start Spark session with local master and 2 cores
spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("parquetTutorial") \
    .getOrCreate()

### Load file and store as parquet

data.csv contains:

```
 0,3375563642,POINT (4.991196 51.11461),2016-10-10 02:00:17
 1,3369963219,POINT (3.991617 50.68576),2016-10-10 02:00:02
 2,74234088,POINT (5.871564 50.54401),2016-10-10 02:00:11
```

In [3]:
#Load from local folder, you have to give complete path preceded by file:///
filename="file:///home/yleborgn/mobiaid/data.csv"
df = spark.read.load(filename, 
                          format='com.databricks.spark.csv', 
                          header='false', 
                          inferSchema='true')

df.printSchema() #Note: types properly inferred

root
 |-- _c0: integer (nullable = true)
 |-- _c1: long (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: timestamp (nullable = true)



In [4]:
df.take(3)

[Row(_c0=0, _c1=3375563642, _c2=u'POINT (4.991196 51.11461)', _c3=datetime.datetime(2016, 10, 10, 2, 0, 17)),
 Row(_c0=1, _c1=3369963219, _c2=u'POINT (3.991617 50.68576)', _c3=datetime.datetime(2016, 10, 10, 2, 0, 2)),
 Row(_c0=2, _c1=74234088, _c2=u'POINT (5.871564 50.54401)', _c3=datetime.datetime(2016, 10, 10, 2, 0, 11))]

### Save to Parquet


In [5]:
filename="file:///home/yleborgn/mobiaid/dataframe.parquet"
%time df.write.mode("overwrite").parquet(filename)

CPU times: user 2.13 ms, sys: 271 µs, total: 2.4 ms
Wall time: 524 ms


### Interact with HDFS

HDFS masternode is at hdfs://master01:8020

In [6]:
#This writes to HDFS
filename="hdfs://master01:8020/user/yleborgn/dataframe.parquet"
%time df.write.mode("overwrite").parquet(filename)

CPU times: user 3.07 ms, sys: 0 ns, total: 3.07 ms
Wall time: 841 ms


In [7]:
#This reads from HDFS Impala/Hive tables:
filename="hdfs://master01:8020/user/hive/warehouse/mlg.db/tab_parquet"
%time df=spark.read.parquet(filename)

CPU times: user 0 ns, sys: 3.36 ms, total: 3.36 ms
Wall time: 231 ms


In [8]:
df.collect()

[Row(gid=0, id=2147483647, geom=bytearray(b'POINT (4.991196 51.11461)'), date_var=datetime.datetime(2016, 10, 10, 4, 0, 17)),
 Row(gid=1, id=2147483647, geom=bytearray(b'POINT (3.991617 50.68576)'), date_var=datetime.datetime(2016, 10, 10, 4, 0, 2)),
 Row(gid=2, id=74234088, geom=bytearray(b'POINT (5.871564 50.54401)'), date_var=datetime.datetime(2016, 10, 10, 4, 0, 11)),
 Row(gid=None, id=None, geom=bytearray(b''), date_var=None)]